In [ ]:
!pip install git+https://github.com/cuongvng/neural-networks-with-PyTorch.git

In [ ]:
!nvidia-smi

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append("../..")
from utils.training_helpers import train_cnn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Load CIFAR dataset

In [ ]:
transform = transforms.Compose([
                                transforms.Resize((96,96)),
                                transforms.ToTensor() # convert data from PIL image to tensor
                                ]) 
cifar_train = torchvision.datasets.CIFAR10(root="../data/", train=True, 
                                     transform=transform, 
                                     target_transform=None, 
                                     download=True)
cifar_test = torchvision.datasets.CIFAR10(root="../data/", train=False, 
                                     transform=transform, 
                                     target_transform=None, 
                                     download=True)

batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=cifar_train,
    batch_size=128,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    dataset=cifar_test,
    batch_size=128,
    shuffle=True,

)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Let's dive into ResNet's architechture.
There are multiple types of ResNet, such as ResNet-18, ResNet-50, ResNet-152, etc., but they has the same core: the **Residual Block**.
In this notebook, I will demonstrate the ResNet-18.

<figure>
    <center><img src="https://github.com/d2l-ai/d2l-en/blob/master/img/resnet-block.svg?raw=1"/></center>
    <center><figcaption class="center">Residual Block</figcaption></center>
</figure>

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1_conv=False):
        super(ResidualBlock, self).__init__()
        self.conv33 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv11 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d()
    
    def forward(self, X):
        X_clone = X.clone()
        
        X = self.conv33(X)
        X = F.relu(self.bn(X))
        X = self.conv33(X)
        X = self.bn(X)
        
        if use_1x1_conv:
            X_clone = self.conv11(X_clone)

        return F.relu(X + X_clone)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [ ]:
n_epochs = 10
train_cnn(net, device, train_loader, test_loader, optimizer, criterion, n_epochs)r